
# 🌸 Iris en Colab: Clasificación con *scikit-learn*

**Objetivo**: completar un flujo supervisado de ML con el dataset *Iris*:
1) carga y exploración, 2) visualización (2D y PCA 3D), 3) *train/test split*,
4) modelos base (Regresión Logística, KNN, SVM), 5) *pipelines* con `StandardScaler` + `PCA`,
6) *GridSearchCV* para ajustar hiperparámetros, 7) evaluación con métricas y curvas de aprendizaje.

> **Entrega**: responde a las preguntas marcadas como **TAREA** en celdas de texto (Markdown) al final de cada sección.


In [ ]:

#@title Instalación y librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, learning_curve, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

plt.rcParams['figure.figsize'] = (7,5)
print('Librerías cargadas.')


## 1) Carga del dataset y descripción

In [ ]:

iris = datasets.load_iris()
X, y = iris.data, iris.target
feature_names = iris.feature_names
target_names = iris.target_names

print(f"Shape X: {X.shape} (150 muestras x 4 características)")
print("Características:", feature_names)
print("Clases:", target_names)

import pandas as pd
df = pd.DataFrame(X, columns=feature_names)
df['target'] = y
df.head()


## 2) Visualización 2D (primeras 2 variables)

In [ ]:
not
_, ax = plt.subplots()
scatter = ax.scatter(X[:, 0], X[:, 1], c=y)
ax.set(xlabel=feature_names[0], ylabel=feature_names[1])
_ = ax.legend(scatter.legend_elements()[0], target_names, loc="lower right", title="Classes")
plt.title("Iris (2 primeras características)")
plt.show()


## 3) PCA 3D (primeras 3 componentes)

In [ ]:

# unused but required import for doing 3d projections with matplotlib < 3.2
import mpl_toolkits.mplot3d  # noqa: F401

fig = plt.figure(1, figsize=(8, 6))
ax = fig.add_subplot(111, projection="3d", elev=-150, azim=110)

X_reduced = PCA(n_components=3).fit_transform(X)
ax.scatter(X_reduced[:, 0], X_reduced[:, 1], X_reduced[:, 2], c=y, s=40)
ax.set_title("First three PCA dimensions")
ax.set_xlabel("1st Eigenvector"); ax.xaxis.set_ticklabels([])
ax.set_ylabel("2nd Eigenvector"); ax.yaxis.set_ticklabels([])
ax.set_zlabel("3rd Eigenvector"); ax.zaxis.set_ticklabels([])
plt.show()


## 4) Train/Test split

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)
X_train.shape, X_test.shape


## 5) Modelos base: Regresión Logística, KNN, SVM (lineal)

In [ ]:

models = {
    "LogReg": LogisticRegression(max_iter=1000),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "SVM-linear": SVC(kernel="linear")
}

results = {}
for name, clf in models.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    print(f"{name:10s} Accuracy: {acc:.3f}")

import pandas as pd
pd.Series(results, name="Accuracy").sort_values(ascending=False)


In [ ]:

best_name = max(results, key=results.get)
best_model = models[best_name]
y_pred = best_model.predict(X_test)

print(f"Mejor modelo: {best_name}")
print(classification_report(y_test, y_pred, target_names=target_names))

cm = confusion_matrix(y_test, y_pred, labels=[0,1,2])
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_names)
disp.plot(values_format="d")
plt.title(f"Matriz de confusión - {best_name}")
plt.show()


## 6) Pipeline con `StandardScaler` + `PCA` + `SVC`

In [ ]:

pipe_svc = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=2)),
    ("svc", SVC())
])

param_grid = {
    "pca__n_components": [2, 3, 4],
    "svc__kernel": ["linear", "rbf"],
    "svc__C": [0.1, 1, 10, 100],
    "svc__gamma": ["scale", 0.01, 0.1, 1]
}

from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(pipe_svc, param_grid, cv=cv, scoring="accuracy", n_jobs=-1)
grid.fit(X_train, y_train)

print("Mejor combinación:", grid.best_params_)
print("Accuracy CV (mejor):", grid.best_score_)
print("Accuracy en test:", accuracy_score(y_test, grid.predict(X_test)))


## 7) Búsqueda simple para KNN

In [ ]:

accs = []
ks = list(range(1, 21))
for k in ks:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=cv, scoring="accuracy")
    accs.append(scores.mean())

plt.plot(ks, accs, marker="o")
plt.xlabel("n_neighbors (k)"); plt.ylabel("Accuracy CV media")
plt.title("Selección de k para KNN (CV)")
plt.show()

k_best = ks[int(np.argmax(accs))]
print("Mejor k:", k_best, "con Accuracy CV:", max(accs))


## 8) Curva de aprendizaje (modelo ganador del GridSearch SVC)

In [ ]:

train_sizes, train_scores, test_scores = learning_curve(
    grid.best_estimator_, X, y, cv=cv, scoring="accuracy",
    train_sizes=np.linspace(0.1, 1.0, 10), n_jobs=-1
)

train_mean = train_scores.mean(axis=1)
test_mean = test_scores.mean(axis=1)

plt.plot(train_sizes, train_mean, marker="o", label="Entrenamiento")
plt.plot(train_sizes, test_mean, marker="s", label="Validación CV")
plt.xlabel("Tamaño de entrenamiento"); plt.ylabel("Accuracy")
plt.title("Curva de aprendizaje - Mejor SVC (pipeline)")
plt.legend()
plt.show()



---

## 🔎 **TAREA — Preguntas para responder**
1. Observando la visualización 2D, ¿qué clase es más separable solo con sépalos? ¿Por qué?
2. En PCA 3D, ¿se distinguen mejor *Versicolor* y *Virginica* respecto a 2D? Comenta.
3. ¿Qué modelo base obtuvo mayor `accuracy`? ¿En qué clases suele confundirse (ver matriz de confusión)?
4. ¿Qué combinación de hiperparámetros del pipeline SVC fue la mejor? Interpreta el rol de `C`, `gamma` y `n_components`.
5. Según la curva de aprendizaje, ¿ves alto *bias* o alto *variance*? ¿Qué harías para mejorar?

> Escribe tus respuestas aquí debajo en Markdown.
